## 텐서보드 사용하기
텐서보드는 학습하는 중간중간 손실값이나 정확도 또는 결과물로 나온 이미지나 사운드 파일을 다양한 방식으로 시각화해 보여줄 수 있음

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data = np.loadtxt('./data.csv', delimiter=',', unpack = True, dtype = 'float32')
# unpack은 행렬의 column을 row로 바꿔주는 함수

In [3]:
# data는 이전 simple_neural_net(classification)과 동일
# 털,날개,기타,포유류,조류 순서로 되어있는 데이터이며(각 row를 의미) 털과 날개는 train에 필요한 data이다.
# 기타,포유류,조류는 각 객체의 라벨을 원-핫-인코딩 한 결과라고 생각하면 된다

In [4]:
print(data)

[[0. 1. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1.]]


In [5]:
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [6]:
# 학습 횟수를 카운트하기 위한 변수(학습이 되면 안되므로 trainable = False)
global_step = tf.Variable(0, trainable = False, name = 'global_step')

In [7]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

텐서보드를 통해 시각화 했을때, 어느 변수가 어디에 쓰이는지 알기 쉽도록 scope와 name을 모두 정의

In [8]:
with tf.name_scope('layer1') :
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name= "W1")
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2') :
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name="W2")
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name="W3")
    model = tf.matmul(L2, W3)

In [9]:
with tf.name_scope("optimizer"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
    #global_step에 앞서 정의한 global_step변수를 넘겨줌으로써 학습을 할 때마다 1씩 증가하게 됨
    train_op = optimizer.minimize(cost, global_step = global_step)
    #손실값을 추적하기 위해 수집할 값을 지정 (histogram, image, audio등 다양한 값을 수집할 수 있음)
    tf.summary.scalar('cost',cost)

In [10]:
sess = tf.Session()

#saver로 앞서 정의한 변수들을 모두 가져와 파일에 저장
saver = tf.train.Saver(tf.global_variables())

#check point를 통해 기존에 학습되어있는 모델이 있다면 restore로 다시 저장, 없다면 변수를 초기화하여 저장
check_point = tf.train.get_checkpoint_state('./model')
if check_point and tf.train.checkpoint_exists(check_point.model_checkpoint_path):
    saver.restore(sess, check_point.model_checkpoint_path)
else :
    sess.run(tf.global_variables_initializer())

merge_all 함수로 지정한 텐서들을 수집한 후, FileWriter 함수를 이용해 그래프와 텐스들의 값을 저장할 디렉토리를 설정

In [11]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [12]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    print('Step: %d, ' % sess.run(global_step),
         'Cost: %.3f, ' % sess.run(cost,feed_dict={X: x_data, Y: y_data}))
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step = sess.run(global_step)) #global_step 변수를 통해 어디서 수집된 것인지를 기록

Step: 1,  Cost: 1.034, 
Step: 2,  Cost: 0.957, 
Step: 3,  Cost: 0.896, 
Step: 4,  Cost: 0.844, 
Step: 5,  Cost: 0.801, 
Step: 6,  Cost: 0.765, 
Step: 7,  Cost: 0.733, 
Step: 8,  Cost: 0.707, 
Step: 9,  Cost: 0.685, 
Step: 10,  Cost: 0.667, 
Step: 11,  Cost: 0.652, 
Step: 12,  Cost: 0.639, 
Step: 13,  Cost: 0.628, 
Step: 14,  Cost: 0.618, 
Step: 15,  Cost: 0.609, 
Step: 16,  Cost: 0.602, 
Step: 17,  Cost: 0.595, 
Step: 18,  Cost: 0.589, 
Step: 19,  Cost: 0.584, 
Step: 20,  Cost: 0.580, 
Step: 21,  Cost: 0.576, 
Step: 22,  Cost: 0.573, 
Step: 23,  Cost: 0.570, 
Step: 24,  Cost: 0.567, 
Step: 25,  Cost: 0.565, 
Step: 26,  Cost: 0.564, 
Step: 27,  Cost: 0.562, 
Step: 28,  Cost: 0.561, 
Step: 29,  Cost: 0.560, 
Step: 30,  Cost: 0.559, 
Step: 31,  Cost: 0.558, 
Step: 32,  Cost: 0.557, 
Step: 33,  Cost: 0.556, 
Step: 34,  Cost: 0.556, 
Step: 35,  Cost: 0.555, 
Step: 36,  Cost: 0.555, 
Step: 37,  Cost: 0.554, 
Step: 38,  Cost: 0.554, 
Step: 39,  Cost: 0.554, 
Step: 40,  Cost: 0.553, 
Step: 41,

In [13]:
# model 저장
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-100'

In [14]:
# 예측, 정확도 확인
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1) #one-hot-encoding이므로 가자 큰 값(1)이 label
print("예측값:", sess.run(prediction, feed_dict={X: x_data}))
print("실제값:", sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("정확도: %.2f" %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


## 텐서보드 확인하기
터미널에 tensorboard --logdir = ./logs 라고 입력